In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the dataset

In [3]:
train_path = r"D:\Swathi\Car colour\train"
val_path = r"D:\Swathi\Car colour\val"
test_path = r"D:\Swathi\Car colour\test"

In [42]:
# Preprocess data

In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_gen = datagen.flow_from_directory(
    val_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_gen = datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 7267 images belonging to 15 classes.
Found 1550 images belonging to 15 classes.
Found 1556 images belonging to 15 classes.


In [40]:
# Create the model using transfer learning

In [7]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
# Train the model

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=callbacks
)


C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - accuracy: 0.1370 - loss: 2.6787 - val_accuracy: 0.2103 - val_loss: 2.4507 - learning_rate: 1.0000e-05
Epoch 2/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 215s 943ms/step - accuracy: 0.4506 - loss: 1.9364 - val_accuracy: 0.3968 - val_loss: 1.9779 - learning_rate: 1.0000e-05
Epoch 3/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 216s 946ms/step - accuracy: 0.5947 - loss: 1.3920 - val_accuracy: 0.4916 - val_loss: 1.5872 - learning_rate: 1.0000e-05
Epoch 4/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 216s 947ms/step - accuracy: 0.6914 - loss: 1.0697 - val_accuracy: 0.5684 - val_loss: 1.2953 - learning_rate: 1.0000e-05
Epoch 5/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 216s 946ms/step - accuracy: 0.7412 - loss: 0.8809 - val_accuracy: 0.6465 - val_loss: 1.0892 - learning_rate: 1.0000e-05
Epoch 6/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 220s 965ms/step - accuracy: 0.7737 - loss: 0.7451 - val_accuracy: 0.6781 - val_loss: 0.9607 - learning_rate: 1.0000e-05
Epoch 7/30
228/228 ━━━━━━━━━━━━━━━━━━━━ 222

In [36]:
# Evaluate the model

In [11]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy*100:.2f}%")

predictions = model.predict(test_gen)
y_pred = np.argmax(predictions, axis=1)
y_true = test_gen.classes

print("Classification Report")
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))

print("Confusion Matrix")
print(confusion_matrix(y_true, y_pred))

49/49 ━━━━━━━━━━━━━━━━━━━━ 43s 884ms/step - accuracy: 0.7402 - loss: 0.7003
Test Accuracy: 77.31%
49/49 ━━━━━━━━━━━━━━━━━━━━ 30s 560ms/step
Classification Report
              precision    recall  f1-score   support

       beige       0.47      0.53      0.50        90
       black       0.78      0.83      0.80        87
        blue       0.90      0.94      0.92       159
       brown       0.70      0.70      0.70       121
        gold       0.63      0.42      0.51        45
       green       0.89      0.83      0.86       121
        grey       0.58      0.58      0.58        92
      orange       0.93      0.88      0.90       114
        pink       0.89      0.89      0.89       103
      purple       0.82      0.67      0.74       115
         red       0.85      0.93      0.89       136
      silver       0.60      0.58      0.59        77
         tan       0.48      0.51      0.50        86
       white       0.82      0.84      0.83        86
      yellow       0.91    

In [34]:
# Save the model

In [13]:
model.save("saved_model/car_color_classifier.h5")